# Exercice 1 Réalisation d’un outil pour dessiner des réseaux

Vous devez réaliser un logiciel permettant de dessiner des architectures réseau de façon
simple et élégante. 
Le cahier des charges imposé par le client indique que votre logiciel
doit permettre de représenter :
- des switchs
- des clients (PC ou téléphone)
- des routeurs

Les différents éléments peuvent être présents de multiple fois dans l’interface. 
La création d’un nouvel item pourra se faire par un raccourci clavier ( C client, S switch, R routeur, par exemple ) ou par toolbar / menu, et il doit être possible de déplacer ou d’effacer les items à la souris.

Un clic droit sur un item doit permettre d’en modifier les propriétés :
- Nom
- Icône

Enfin, il doit être possible de dessiner des traits sur le schémas afin de représenter les liens réseau.

In [ ]:
import tkinter as tk
from tkinter import simpledialog
from PIL import Image, ImageTk

class WishPacketTracer:
    def __init__(self, root):
        self.root = root
        self.root.title("Wish Packet Tracer")

        # 1) 
        self.canvas = tk.Canvas(root, bg="ivory", width=800, height=600)
        self.canvas.pack(expand=tk.YES, fill=tk.BOTH)

# 1) 
root = tk.Tk()
app = WishPacketTracer(root)
root.mainloop()

J'ai dans mon premier commit, créer une classe qui permettra d'instancier les objets de l'application car il doivent être présents plusieurs fois en même temps dans la fenêtre.
A la fin de mon programme, je crée une fenêtre qui s'appelle root et à laquelle je créer une instance de la classe "WishPacketTracer".
Et donc le programme va créer un canvas basique de 800x600 qui va s'étendre càd prendre la place dans tout son espace (en l'occurence la fenêtre) et doit s'élargir horizontalement et verticalement à la fois.

<img src="./img/i1.png">

Puis j'ai ajouté à mon code, une barre d'outils dans laquelle, j'ai au préalable chargé mes 3 images (pc,routeur et switch) grâce à PIL et je les redimensionne et je les ai rendues cliquable en les affichant en tant que Button dans ma barre d'outils. Mes 3 images vont alors s'afficher au clic dans le canvas.

# Exercice 2 Des traits bien droits !
Faites en sorte que les lignes ne puissent être que horizontales ou verticales lorsque
l’utilisateur maintient la touche CTRL enfoncée.


# Exercice 3 Un client exigeant ...
Le client, satisfait par votre travail précédant, demande à ce que vous alliez plus loin. Il
souhaite maintenant que les items aient des ports sur lesquels votre souris sera aimantée
lorsque vous dessinez des traits. Le nombre de ports des routeurs et des switchs sera
paramétrable de 1 à 4. Les clients, eux, auront toujours un seul port